In [1]:
import sys
sys.executable

'/root/home/envforir/bin/python'

In [2]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders.base import BaseLoader
from langchain.schema import Document
import json
import re
import numpy as np
from typing import List
from langchain.text_splitter import TextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from dotenv import load_dotenv
import os
import pickle
from langchain_community.vectorstores import FAISS
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains.query_constructor.base import AttributeInfo, get_query_constructor_prompt
from langchain.retrievers.self_query.base import SelfQueryRetriever
import numpy as np
import faiss
import json
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from langchain.docstore.document import Document
from langchain.document_loaders.base import BaseLoader



USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
load_dotenv() 
openai_api_key = os.getenv("OPENAI_API_KEY")

In [78]:
# class JSONLLoader(BaseLoader):
#     def __init__(self, file_path: str):
#         self.file_path = file_path

#     def load(self):
#         documents = []
#         seq_num = 1
        
#         with open(self.file_path, 'r', encoding='utf-8') as file:
#             for line in file:
#                 data = json.loads(line)
                
#                 doc = Document(
#                     page_content=data['content'],
#                     metadata={
#                         'docid': data['docid'],
#                         'src': data.get('src', ''),
#                         'source': self.file_path,
#                         'seq_num': seq_num,
#                     }
#                 )
#                 documents.append(doc)
#                 seq_num += 1
        
#         return documents

# file_path = "/data/ephemeral/home/upstage-ai-final-ir2/HM/data/documents.jsonl"
# loader = JSONLLoader(file_path)
# documents = loader.load()

# print(f"문서의 수: {len(documents)}")

문서의 수: 4272


# ElasticSearch 설정 

In [8]:
# 데몬 구동 확인 -> password 설정 
!ps -ef | grep elasticsearch

iruser   1050670       1 16 10:06 pts/10   00:00:53 /home/iruser/elasticsearch-8.8.0/jdk/bin/java -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -Djava.security.manager=allow -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Dlog4j2.formatMsgNoLookups=true -Djava.locale.providers=SPI,COMPAT --add-opens=java.base/java.io=org.elasticsearch.preallocate -XX:+UseG1GC -Djava.io.tmpdir=/tmp/elasticsearch-6327256372733567408 -XX:+HeapDumpOnOutOfMemoryError -XX:+ExitOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,level,pid,tags:filecount=32,filesize=64m -Xms30720m -Xmx30720m -XX:MaxDirectMemorySize=16106127360 -XX:InitiatingHeapOccupancyPercent=30 

In [7]:
es_username = 'elastic'
es_password = 'gHlnDi-tV2dYFt37_0gd'

# ElasticSearch Client 생성 
es = Elasticsearch(['https://localhost:9200'], basic_auth=(es_username, es_password), ca_certs="/data/ephemeral/home/upstage-ai-final-ir2/HM/elasticsearch-8.8.0/config/certs/http_ca.crt", verify_certs=False)

# ElasticSearch Client 정보 확인 
es.info()

/root/home/envforir/lib/python3.10/site-packages/elasticsearch/_sync/client/__init__.py:395: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ObjectApiResponse({'name': 'instance-8909', 'cluster_name': 'elasticsearch', 'cluster_uuid': 'r4Ih9u1sTfKbhPDz61kqLQ', 'version': {'number': '8.8.0', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': 'c01029875a091076ed42cdb3a41c10b1a9a5a20f', 'build_date': '2023-05-23T17:16:07.179039820Z', 'build_snapshot': False, 'lucene_version': '9.6.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [8]:
# 색인을 위한 settings 설정 
settings = { "analysis": {"analyzer": {"nori": {"type": "custom",  #>> custom: 사용자 정의 분석기임을 명시 
                                                "tokenizer": "nori_tokenizer",
                                                "decompound_mode": "mixed",  # 복합 명사를 분해하는 방식을 설정. mixed: 복합명사를 분해하되, 분해된 토큰과 원본 토큰을 모두 저장 
                                                "filter": ["nori_posfilter"]}},
                          "filter": {"nori_posfilter":{"type": "nori_part_of_speech",
                                                       "stoptags": ["E", "J", "SC", "SE", "SF", "VCN", "VCP", "VX"]}}}  # 제거할 품사 태그를 지정 
            }

# 색인을 위한 mapping 설정(역색인 필드, 임베딩 필드 모두 설정)
mappings = { "properties": {"content": {"type": "text",
                                        "analyzer": "nori"},
                            "embeddings": {"type": "dense_vector",
                                           "dims": 768,
                                           "index": True,
                                           "similarity": "l2_norm"}}  # 벡터 간 유사도 계산 방식: L2 norm 
            }


In [9]:
# 새로운 index 생성 
def create_es_index(index, settings, mappings):
    if es.indices.exists(index=index):   # 만약 인덱스가 이미 존재한다면 
        es.indices.delete(index=index)   # 새로운 것으로 설정하기 위해 삭제 
    es.indices.create(index=index, settings=settings, mappings=mappings)  # 지정된 settings로 새로운 인덱스 생성 

# 지정된 인덱스 삭제 
def delete_es_index(index):
    es.indices.delete(index=index)


In [10]:
  # ElasticSearch 헬퍼 함수를 사용하여 대량 인덱싱 수행 
def bulk_add(index, docs):
    actions = [{'_index': index, '_source': doc} for doc in docs]
    return helpers.bulk(es, actions)

In [11]:
# 'test' 인덱스 생성
create_es_index("test", settings, mappings)

/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [12]:
# 문서 데이터 읽어들이기
documents = []
with open('/data/ephemeral/home/upstage-ai-final-ir2/upstage-ai-final-ir2/HM/data/documents.jsonl', 'r') as f:
    for line in f:
        document = json.loads(line)
        documents.append(document)

In [14]:
# 문서 데이터 Elasticsearch에 인덱싱
bulk_add('test', documents)

/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/root/home/envforir/lib/python3.10/site-packages/urllib3/c

(4272, [])

In [21]:
# 유사도 검색 테스트 
query = "나무의 분류에 대해 조사해 보기 위한 방법은?"

In [22]:
# OpenAI LLM 초기화 
llm = OpenAI(temperature=0)
#>> GPT-3.5-turbo

In [23]:
# Standalone Query Generator 프롬프트 템플릿
standalone_query_prompt = PromptTemplate(
    input_variables=["question"],
    template="""질문 query를 요약하려고 합니다. 핵심내용을 포함한 주제를 출력해주세요. 하나의 구문으로 출력해주세요. 
    아래는 예시입니다. 
    
    원래의 질문: "금성이 밝게 보이는 이유가 뭐야?"
    생성할 standalone_query: "금성 밝기 원인"

    원래의 질문: {question}
    독립적인 질문:
    """
)

# Standalone Query Generator 체인 생성
standalone_query_chain = LLMChain(llm=llm, prompt=standalone_query_prompt)

# Standalone Query 생성
standalone_query = standalone_query_chain.run(query)

print(f"Original Query: {query}")
print(f"Standalone Query: {standalone_query}")

Original Query: 나무의 분류에 대해 조사해 보기 위한 방법은?
Standalone Query: 
    나무 분류 조사 방법


# 예제 테스트 

In [24]:
# 역색인을 이용한 검색 
def sparse_retrieve(query_str, size):
    query = { "match": {"content": {"query": query_str}}}
    return es.search(index='test', query=query, size=size, sort='_score')
#>> query_str: 검색어
#>> size: 검색 결과 개수 
#>> 'test' 인덱스에서 정의된 쿼리를 실행 -> 검색 결과를 size개 만큼 반환 

In [38]:
# 역색인을 사용하는 검색 예제 
search_result_retrieve = sparse_retrieve(query, 10)

# 결과 출력 테스트
for rst in search_result_retrieve['hits']['hits']:
    print("검색 결과:", search_result_retrieve)
    print('score:', rst['_score'], 'source:', rst['_source']['content'])

검색 결과: {'took': 3, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 4238, 'relation': 'eq'}, 'max_score': 20.857122, 'hits': [{'_index': 'test', '_id': 'dV_WZ5ABhqTPfro3H_yb', '_score': 20.857122, '_source': {'docid': 'c63b9e3a-716f-423a-9c9b-0bcaa1b9f35d', 'src': 'ko_ai2_arc__ARC_Challenge__test', 'content': '한 학생이 다양한 종류의 나무를 조사하고 있습니다. 이 학생은 성장 속도, 온도 범위, 크기가 비슷한 두 나무를 발견했습니다. 그러나 이 두 나무의 잎과 꽃은 서로 다릅니다. 이러한 특징을 고려하면, 이 나무들은 대체로 같은 속에 속해 있을 것으로 추측됩니다. 같은 속에 속한 나무들은 종류별로 유사한 특징을 가지고 있으며, 이는 생물 분류학에서 중요한 기준 중 하나입니다. 따라서 이 학생의 조사 결과는 나무의 분류와 관련된 중요한 정보를 제공할 수 있습니다. 이러한 조사는 나무의 성장과 생태에 대한 이해를 높이는 데 도움이 될 것입니다.'}}, {'_index': 'test', '_id': 'lF_WZ5ABhqTPfro3Hvqt', '_score': 16.680271, '_source': {'docid': '9712bdf6-9419-4953-a8f1-8a4015dee986', 'src': 'ko_ai2_arc__ARC_Challenge__train', 'content': '생물학에서 일부 생물체의 분류 방법이 변경되었습니다. 이제 생물체를 재분류하는 데에는 구조보다는 분자 수준에서의 조사가 사용됩니다. 이 새로운 방법은 생물체의 유전자나 단백질의 구조와 기능을 분석하여 그들의 진화적 관계를 밝

/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [27]:
# Document 객체 변환: 딕셔너리 리스트인 documents를 객체 리스트로 
# FAISS.from_documents() 메서드는 'Document' 객체의 리스트를 기대 

documents = [Document(page_content=doc['content'], metadata={"source": doc.get('src', ''), "docid": doc.get('docid', '')}) for doc in documents]


In [28]:
vectorstore = FAISS.from_documents(documents=documents, embedding=OpenAIEmbeddings())
#>> 임베딩 + 인덱싱 + 벡터스토어 생성 

In [29]:
# K개만 검색하여 결과를 반환
K = 3
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": K})
search_result = retriever.get_relevant_documents(standalone_query)

# 결과 출력
for i, doc in enumerate(search_result, 1):
    print(f"\n문서 {i}:")
    print(f"내용: {doc.page_content[:100]}...")  # 처음 100자만 출력
    print(f"메타데이터: {doc.metadata}")
    print("---")


/root/home/envforir/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(



문서 1:
내용: 한 학생이 다양한 종류의 나무를 조사하고 있습니다. 이 학생은 성장 속도, 온도 범위, 크기가 비슷한 두 나무를 발견했습니다. 그러나 이 두 나무의 잎과 꽃은 서로 다릅니다. 이러...
메타데이터: {'source': 'ko_ai2_arc__ARC_Challenge__test', 'docid': 'c63b9e3a-716f-423a-9c9b-0bcaa1b9f35d'}
---

문서 2:
내용: 고든 여사의 수업은 자연과학에 관한 것입니다. 이 수업에서는 단풍나무에 대해 공부합니다. 단풍나무는 가을에 아름다운 빨간색으로 변하는 나무로 유명합니다. 학생들은 이 나무의 특성을...
메타데이터: {'source': 'ko_ai2_arc__ARC_Challenge__train', 'docid': 'b730a81a-3903-42ca-9633-88b0ebb9eb42'}
---

문서 3:
내용: 학생이 다른 토양 유형에서 식물의 성장을 연구했습니다. 연구에서는 날짜와 식물의 높이, 그리고 토양 유형을 기록하였습니다. 이러한 데이터를 정리하는 가장 좋은 방법은 날짜와 식물의...
메타데이터: {'source': 'ko_ai2_arc__ARC_Challenge__test', 'docid': 'a153c822-be9f-4346-8558-34365ed7b4f0'}
---


In [30]:
# 두 개 하나로 합치기(sparse+dense)
def search_documents(query, sparse_k=50, dense_k=3):
    # Standalone Query 생성
    standalone_query = standalone_query_chain.run(query)
    print(f"Original Query: {query}")
    print(f"Standalone Query: {standalone_query}")

    # sparse_retrieve로 10개 선정
    sparse_results = sparse_retrieve(standalone_query, sparse_k)

    # sparse 결과를 Document 객체로 변환
    documents = []
    for hit in sparse_results['hits']['hits']:
        content = hit['_source']['content']
        metadata = {'score': hit['_score']}
        documents.append(Document(page_content=content, metadata=metadata))

    # 벡터 유사도 검색으로 최종 3개 선정
    final_results = vectorstore.similarity_search(standalone_query, k=dense_k)

    # 결과 출력
    for i, doc in enumerate(final_results, 1):
        print(f"\n문서 {i}:")
        print(f"내용: {doc.page_content[:100]}...")  # 처음 100자만 출력
        print(f"메타데이터: {doc.metadata}")
        print("---")

    return final_results

In [31]:
# 검색 실행
query = "나무의 분류에 대해 조사해 보기 위한 방법은?"
search_documents(query)

Original Query: 나무의 분류에 대해 조사해 보기 위한 방법은?
Standalone Query: 
    나무 분류 조사 방법


/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(



문서 1:
내용: 한 학생이 다양한 종류의 나무를 조사하고 있습니다. 이 학생은 성장 속도, 온도 범위, 크기가 비슷한 두 나무를 발견했습니다. 그러나 이 두 나무의 잎과 꽃은 서로 다릅니다. 이러...
메타데이터: {'source': 'ko_ai2_arc__ARC_Challenge__test', 'docid': 'c63b9e3a-716f-423a-9c9b-0bcaa1b9f35d'}
---

문서 2:
내용: 고든 여사의 수업은 자연과학에 관한 것입니다. 이 수업에서는 단풍나무에 대해 공부합니다. 단풍나무는 가을에 아름다운 빨간색으로 변하는 나무로 유명합니다. 학생들은 이 나무의 특성을...
메타데이터: {'source': 'ko_ai2_arc__ARC_Challenge__train', 'docid': 'b730a81a-3903-42ca-9633-88b0ebb9eb42'}
---

문서 3:
내용: 학생이 다른 토양 유형에서 식물의 성장을 연구했습니다. 연구에서는 날짜와 식물의 높이, 그리고 토양 유형을 기록하였습니다. 이러한 데이터를 정리하는 가장 좋은 방법은 날짜와 식물의...
메타데이터: {'source': 'ko_ai2_arc__ARC_Challenge__test', 'docid': 'a153c822-be9f-4346-8558-34365ed7b4f0'}
---


[Document(page_content='한 학생이 다양한 종류의 나무를 조사하고 있습니다. 이 학생은 성장 속도, 온도 범위, 크기가 비슷한 두 나무를 발견했습니다. 그러나 이 두 나무의 잎과 꽃은 서로 다릅니다. 이러한 특징을 고려하면, 이 나무들은 대체로 같은 속에 속해 있을 것으로 추측됩니다. 같은 속에 속한 나무들은 종류별로 유사한 특징을 가지고 있으며, 이는 생물 분류학에서 중요한 기준 중 하나입니다. 따라서 이 학생의 조사 결과는 나무의 분류와 관련된 중요한 정보를 제공할 수 있습니다. 이러한 조사는 나무의 성장과 생태에 대한 이해를 높이는 데 도움이 될 것입니다.', metadata={'source': 'ko_ai2_arc__ARC_Challenge__test', 'docid': 'c63b9e3a-716f-423a-9c9b-0bcaa1b9f35d'}),
 Document(page_content='고든 여사의 수업은 자연과학에 관한 것입니다. 이 수업에서는 단풍나무에 대해 공부합니다. 단풍나무는 가을에 아름다운 빨간색으로 변하는 나무로 유명합니다. 학생들은 이 나무의 특성을 배우고, 그 중에서도 미터 자로 측정할 수 있는 특성에 대해 알아봅니다. 미터 자는 길이를 측정하는 도구로, 학생들은 단풍나무의 나뭇가지의 길이를 미터 자를 사용하여 측정할 수 있습니다. 이를 통해 학생들은 단풍나무의 나뭇가지의 길이에 대해 정확한 정보를 얻을 수 있습니다. 이러한 과정을 통해 학생들은 과학적인 방법으로 단풍나무에 대해 이해하고, 미터 자를 사용하여 길이를 측정하는 기술을 익힐 수 있습니다. 고든 여사의 수업은 학생들에게 자연과학의 중요성을 깨닫게 하고, 측정과 관찰을 통해 지식을 확장하는 능력을 키워줍니다.', metadata={'source': 'ko_ai2_arc__ARC_Challenge__train', 'docid': 'b730a81a-3903-42ca-9633-88b0ebb9eb42'}),
 Document(page_content='학생이 다른 토양

In [32]:
# RAG 구현에 필요한 Question Answering을 위한 LLM  프롬프트
prompt = hub.pull("rlm/rag-prompt")
chat = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

In [36]:
def format_docs(docs):
    global references
    references = docs
    return '\n\n'.join(doc.page_content for doc in docs)

In [33]:
# def search_documents(query, eval_id, sparse_k=30, dense_k=3):
#     response = {
#         "eval_id": eval_id,
#         "standalone_query": "",
#         "topk": [],
#         "answer": "",
#         "references": []
#     }

#     # Standalone Query 생성
#     standalone_query = standalone_query_chain.run(query)
#     response["standalone_query"] = standalone_query
#     print(f"Original Query: {query}")
#     print(f"Standalone Query: {standalone_query}")

#     # sparse_retrieve로 선정
#     sparse_results = sparse_retrieve(standalone_query, sparse_k)

#     # sparse 결과를 Document 객체로 변환
#     documents = []
#     for hit in sparse_results['hits']['hits']:
#         content = hit['_source']['content']
#         metadata = {'score': hit['_score'], 'docid': hit['_id']}
#         documents.append(Document(page_content=content, metadata=metadata))

#     # 벡터 유사도 검색으로 최종 dense_k개 선정
#     final_results = vectorstore.similarity_search(standalone_query, k=dense_k)

#     # 결과 정리
#     for doc in final_results:
#         response["topk"].append(doc.metadata['docid'])
#         response["references"].append(doc.page_content)

#     # RAG를 위한 context 생성
#     context = format_docs(final_results)

#     # LLM을 사용하여 답변 생성
#     rag_prompt = PromptTemplate(
#         template="Context: {context}\n\nQuestion: {question}\n\nAnswer:",
#         input_variables=["context", "question"]
#     )
    
#     rag_chain = LLMChain(llm=chat, prompt=rag_prompt)
    
#     response["answer"] = rag_chain.run(context=context, question=query)

#     return response

In [39]:
def is_science_question(query):
    prompt = PromptTemplate(
        input_variables=["question"],
        template="다음 질문이 과학 상식에 관한 것인지 판단해주세요. '예' 또는 '아니오'로만 답변해주세요.\n\n질문: {question}\n\n답변:"
    )
    chain = LLMChain(llm=chat, prompt=prompt)
    response = chain.run(question=query)
    return response.strip().lower() == "예"

def search_documents(query, eval_id, sparse_k=30, dense_k=3):
    response = {
        "eval_id": eval_id,
        "standalone_query": "",
        "topk": [],
        "answer": "",
        "references": []
    }

    # 과학 상식 질문인지 판단
    if not is_science_question(query):
        response["answer"] = "이 질문은 과학 상식에 관한 것이 아닙니다."
        return response

    # Standalone Query 생성
    standalone_query = standalone_query_chain.run(query)
    response["standalone_query"] = standalone_query
    print(f"Original Query: {query}")
    print(f"Standalone Query: {standalone_query}")

    # sparse_retrieve로 선정
    sparse_results = sparse_retrieve(standalone_query, sparse_k)

    # sparse 결과를 Document 객체로 변환
    documents = []
    for hit in sparse_results['hits']['hits']:
        content = hit['_source']['content']
        metadata = {'score': hit['_score'], 'docid': hit['_id']}
        documents.append(Document(page_content=content, metadata=metadata))

    # 벡터 유사도 검색으로 최종 dense_k개 선정
    final_results = vectorstore.similarity_search(standalone_query, k=dense_k)

    # 결과 정리
    for doc in final_results:
        response["topk"].append(doc.metadata['docid'])
        response["references"].append(doc.page_content)

    # RAG를 위한 context 생성
    context = format_docs(final_results)

    # LLM을 사용하여 답변 생성
    rag_prompt = PromptTemplate(
        template="Context: {context}\n\nQuestion: {question}\n\nAnswer:",
        input_variables=["context", "question"]
    )
    
    rag_chain = LLMChain(llm=chat, prompt=rag_prompt)
    
    response["answer"] = rag_chain.run(context=context, question=query)

    return response

In [34]:
def eval_rag(eval_filename, output_filename):
    with open(eval_filename) as eval_lines, open(output_filename, 'w') as output_lines:
        for idx, eval_line in enumerate(eval_lines):
            j = json.loads(eval_line)
            print(f'Test {idx}\nQuestion: {j["msg"]}')
            response = search_documents(j["msg"], j["eval_id"])
            print(f'Answer: {response["answer"]}\n')
            output_lines.write(f'{json.dumps(response, ensure_ascii=False)}\n')


In [37]:
# 평가 데이터에 대해서 결과 생성 - 파일 포맷은 jsonl이지만 파일명은 csv 사용
eval_rag('../data/eval.jsonl', '.sample_submission_9.csv')

Test 0
Question: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]
Original Query: [{'role': 'user', 'content': '나무의 분류에 대해 조사해 보기 위한 방법은?'}]
Standalone Query: "나무 분류 조사 방법"


/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Answer: 나무의 분류에 대해 조사하기 위한 방법은 나무의 성장 속도, 온도 범위, 크기, 잎과 꽃의 특징 등을 고려하여 같은 속에 속해 있는지를 확인하는 것입니다. 같은 속에 속한 나무들은 종류별로 유사한 특징을 가지고 있으며, 이는 생물 분류학에서 중요한 기준 중 하나입니다. 따라서 이러한 특징을 고려하여 나무의 분류를 확인할 수 있습니다.

Test 1
Question: [{'role': 'user', 'content': '각 나라에서의 공교육 지출 현황에 대해 알려줘.'}]
Original Query: [{'role': 'user', 'content': '각 나라에서의 공교육 지출 현황에 대해 알려줘.'}]
Standalone Query: "각 나라의 공교육 지출 현황"


/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Answer: 현재 전세계의 공공 교육 지출은 세계 GDP의 약 4%를 차지하고 있습니다. 이는 국가들이 교육에 많은 자원을 투자하고 있다는 것을 보여줍니다. 국가들은 교육을 통해 인재를 양성하고, 인구의 교육 수준을 높여 경제적인 경쟁력을 강화하고자 합니다. 이러한 투자는 국가의 교육 체계를 강화하고, 국민들의 교육 기회를 확대하는 데 도움을 줍니다.

Test 2
Question: [{'role': 'user', 'content': '기억 상실증 걸리면 너무 무섭겠다.'}, {'role': 'assistant', 'content': '네 맞습니다.'}, {'role': 'user', 'content': '어떤 원인 때문에 발생하는지 궁금해.'}]
Original Query: [{'role': 'user', 'content': '기억 상실증 걸리면 너무 무섭겠다.'}, {'role': 'assistant', 'content': '네 맞습니다.'}, {'role': 'user', 'content': '어떤 원인 때문에 발생하는지 궁금해.'}]
Standalone Query: "기억 상실증 원인"


/root/home/envforir/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


KeyboardInterrupt: 